# `auto_arima`

Pyramid bring R's [`auto.arima`](https://www.rdocumentation.org/packages/forecast/versions/7.3/topics/auto.arima) functionality to Python by wrapping statsmodel [`ARIMA`](https://github.com/statsmodels/statsmodels/blob/master/statsmodels/tsa/arima_model.py) and [`SARIMAX`](https://github.com/statsmodels/statsmodels/blob/master/statsmodels/tsa/statespace/sarimax.py) models into a singular scikit-learn-esque estimator ([`pyramid.arima.ARIMA`](https://github.com/tgsmith61591/pyramid/blob/master/pyramid/arima/arima.py)) and adding several layers of degree and seasonal differencing tests to identify the optimal model parameters.

__Pyramid ARIMA models:__

  - Are fully picklable for easy persistence and model deployment
  - Can handle seasonal terms (unlike statsmodels ARIMAs)
  - Follow sklearn model fit/predict conventions

In [1]:
import numpy as np
import pyramid

print('numpy version: %r' % np.__version__)
print('pyramid version: %r' % pyramid.__version__)

numpy version: '1.11.3'
pyramid version: '0.2-dev'


We'll start by defining an array of data from an R time-series, `wineind`:

```r
> forecast::wineind
       Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov   Dec
1980 15136 16733 20016 17708 18019 19227 22893 23739 21133 22591 26786 29740
1981 15028 17977 20008 21354 19498 22125 25817 28779 20960 22254 27392 29945
1982 16933 17892 20533 23569 22417 22084 26580 27454 24081 23451 28991 31386
1983 16896 20045 23471 21747 25621 23859 25500 30998 24475 23145 29701 34365
1984 17556 22077 25702 22214 26886 23191 27831 35406 23195 25110 30009 36242
1985 18450 21845 26488 22394 28057 25451 24872 33424 24052 28449 33533 37351
1986 19969 21701 26249 24493 24603 26485 30723 34569 26689 26157 32064 38870
1987 21337 19419 23166 28286 24570 24001 33151 24878 26804 28967 33311 40226
1988 20504 23060 23562 27562 23940 24584 34303 25517 23494 29095 32903 34379
1989 16991 21109 23740 25552 21752 20294 29009 25500 24166 26960 31222 38641
1990 14672 17543 25453 32683 22449 22316 27595 25451 25421 25288 32568 35110
1991 16052 22146 21198 19543 22084 23816 29961 26773 26635 26972 30207 38687
1992 16974 21697 24179 23757 25013 24019 30345 24488 25156 25650 30923 37240
1993 17466 19463 24352 26805 25236 24735 29356 31234 22724 28496 32857 37198
1994 13652 22784 23565 26323 23779 27549 29660 23356
```

Note that the frequency of the data is 12:

```r
> frequency(forecast::wineind)
[1] 12
```

In [2]:
# this is a dataset from R
wineind = np.array([
    # Jan    Feb    Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov    Dec
    15136, 16733, 20016, 17708, 18019, 19227, 22893, 23739, 21133, 22591, 26786, 29740, 
    15028, 17977, 20008, 21354, 19498, 22125, 25817, 28779, 20960, 22254, 27392, 29945, 
    16933, 17892, 20533, 23569, 22417, 22084, 26580, 27454, 24081, 23451, 28991, 31386, 
    16896, 20045, 23471, 21747, 25621, 23859, 25500, 30998, 24475, 23145, 29701, 34365, 
    17556, 22077, 25702, 22214, 26886, 23191, 27831, 35406, 23195, 25110, 30009, 36242, 
    18450, 21845, 26488, 22394, 28057, 25451, 24872, 33424, 24052, 28449, 33533, 37351, 
    19969, 21701, 26249, 24493, 24603, 26485, 30723, 34569, 26689, 26157, 32064, 38870, 
    21337, 19419, 23166, 28286, 24570, 24001, 33151, 24878, 26804, 28967, 33311, 40226, 
    20504, 23060, 23562, 27562, 23940, 24584, 34303, 25517, 23494, 29095, 32903, 34379, 
    16991, 21109, 23740, 25552, 21752, 20294, 29009, 25500, 24166, 26960, 31222, 38641, 
    14672, 17543, 25453, 32683, 22449, 22316, 27595, 25451, 25421, 25288, 32568, 35110, 
    16052, 22146, 21198, 19543, 22084, 23816, 29961, 26773, 26635, 26972, 30207, 38687, 
    16974, 21697, 24179, 23757, 25013, 24019, 30345, 24488, 25156, 25650, 30923, 37240, 
    17466, 19463, 24352, 26805, 25236, 24735, 29356, 31234, 22724, 28496, 32857, 37198, 
    13652, 22784, 23565, 26323, 23779, 27549, 29660, 23356]
).astype(np.float64)

## Fitting an ARIMA

We will first fit a seasonal ARIMA. Note that you do not need to call `auto_arima` in order to fit a model&mdash;if you know the order and seasonality of your data, you can simply fit an ARIMA with the defined hyper-parameters:

In [3]:
from pyramid.arima import ARIMA

fit = ARIMA(order=(1, 1, 1), seasonal_order=(0, 1, 1, 12)).fit(y=wineind)

Note that your data does not have to exhibit seasonality to work with an ARIMA. We could fit an ARIMA against the same data with no seasonal terms whatsoever (but it is unlikely that it will perform better; quite the opposite, likely).

In [4]:
fit = ARIMA(order=(1, 1, 1), seasonal_order=None).fit(y=wineind)

## Finding the optimal model hyper-parameters using `auto_arima`:

If you are unsure (as is common) of the best parameters for your model, let `auto_arima` figure it out for you. `auto_arima` is similar to an ARIMA-specific grid search. Note that it is possible for `auto_arima` not to find a model that will converge; if this is the case, it will raise a `ValueError`.

`auto_arima` can fit a random search that is much faster than the exhaustive one by enabling `random=True`. If your random search returns too many invalid (nan) models, you might try increasing `n_fits` or making it an exhaustive search.

In [6]:
from pyramid.arima import auto_arima

fit = auto_arima(wineind, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                 start_P=0, seasonal=True, n_jobs=-1, d=1, D=1, trace=True,
                 error_action='ignore',  # don't want to know if an order does not work
                 suppress_warnings=True,  # don't want convergence warnings
                 random=True, random_state=42,  # we can fit a random search (not exhaustive)
                 n_fits=10)

fit.summary()

Fit ARIMA: order=(3, 1, 2) seasonal_order=(1, 1, 1, 12); AIC=nan, BIC=nan
Fit ARIMA: order=(1, 1, 3) seasonal_order=(0, 1, 1, 12); AIC=3068.842, BIC=3091.036
Fit ARIMA: order=(3, 1, 3) seasonal_order=(0, 1, 1, 12); AIC=3072.626, BIC=3101.160
Fit ARIMA: order=(2, 1, 3) seasonal_order=(1, 1, 1, 12); AIC=3071.523, BIC=3100.057
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=3068.503, BIC=3090.696
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=3068.757, BIC=3090.951
Fit ARIMA: order=(3, 1, 3) seasonal_order=(0, 1, 2, 12); AIC=3073.580, BIC=3105.285
Fit ARIMA: order=(1, 1, 1) seasonal_order=(2, 1, 2, 12); AIC=3070.103, BIC=3095.467
Fit ARIMA: order=(3, 1, 3) seasonal_order=(2, 1, 1, 12); AIC=3075.530, BIC=3110.405
Fit ARIMA: order=(1, 1, 3) seasonal_order=(2, 1, 2, 12); AIC=3072.042, BIC=3103.747


Statespace Model Results 
 
 Dep. Variable: y No. Observations: 176 
 
 
 Model: SARIMAX(2, 1, 1)x(0, 1, 2, 12) Log Likelihood -1527.251 
 
 
 Date: Fri, 02 Jun 2017 AIC 3068.503 
 
 
 Time: 14:37:28 BIC 3090.696 
 
 
 Sample: 0 HQIC 3077.504 
 
 
 - 176 
 
 
 Covariance Type: opg 
 
 
 
 
 coef std err z P>|z| [0.025 0.975] 
 
 
 intercept -67.3451 54.539 -1.235 0.217 -174.239 39.549 
 
 
 ar.L1 0.0576 0.133 0.434 0.664 -0.202 0.318 
 
 
 ar.L2 -0.1084 0.079 -1.371 0.170 -0.263 0.047 
 
 
 ma.L1 -0.6350 0.147 -4.325 0.000 -0.923 -0.347 
 
 
 ma.S.L12 -0.3809 0.061 -6.283 0.000 -0.500 -0.262 
 
 
 ma.S.L24 -0.0786 0.073 -1.075 0.283 -0.222 0.065 
 
 
 sigma2 8.283e+06 0.001 1.52e+10 0.000 8.28e+06 8.28e+06 
 
 
 
 
 Ljung-Box (Q): 48.47 Jarque-Bera (JB): 21.32 
 
 
 Prob(Q): 0.17 Prob(JB): 0.00 
 
 
 Heteroskedasticity (H): 1.18 Skew: -0.60 
 
 
 Prob(H) (two-sided): 0.55 Kurtosis: 4.31

## Predicting future values

After your model is fit, you can forecast future values using the `predict` function, just like in sci-kit learn:

In [7]:
next_25 = fit.predict(n_periods=25)
next_25

array([ 21564.24609504,  26148.6233966 ,  30221.36545118,  35073.8875762 ,
        12645.82508259,  19519.61905148,  21178.84100404,  23297.50921846,
        21241.35723196,  23338.57494515,  26687.60871594,  21958.45608919,
        19387.77285058,  23184.97155491,  27371.5250093 ,  32335.49897243,
        10092.26139907,  16402.47208156,  18233.70473774,  20203.90193885,
        18178.13067446,  19880.02006711,  23406.80414459,  19062.51896342,
        16141.87377659])

## Visualizing forecasts

In [8]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [9]:
import pandas as pd

def plot_arima(truth, forecasts, title="Forecasted values", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    forecasts = pd.Series(forecasts, index=np.arange(n_truth, n_truth + n_forecasts))
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(truth.index, truth.values, color=c1, legend='Observed')
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted')
    
    return p
    
# call the plotting func
show(plot_arima(wineind, next_25))

<Bokeh Notebook handle for In[9] >

(Github does not render the Bokeh output, so here's the image that's generated from the above code):

![Bokeh output](quick_start_output.png)